In [5]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import random



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [36]:
def load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=0.1,
                           random_seed = 1,
                           shuffle=True):
    

    noise_type = "" if noise_type == "None" else noise_type
    path = data_dir + "/" + noise_type  + "/" + data_type
    print("path: ", path)
    dataset = ImageFolder(root=path, transform=transform)
  

    num_samples = len(dataset)
    print("length of original dataset:", num_samples)
    indices = list(range(num_samples)) 
    if data_percentage != 1:
        needed_length = int(num_samples*data_percentage)
        random.shuffle(indices)
        indices = indices[:needed_length]
    print(len(indices))
    

   

In [37]:
data_dir = '../data/vegetable_images' 
batch_size = 64
data_type = "test"
noise_type = "gaussian_noise"
noise_percentage = 100                    
transform = transforms.Compose([
    transforms.Resize((227, 227)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])                   
data_percentage = 0.1


load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=data_percentage,
                           random_seed = 1,
                           shuffle=True)

path:  ../data/vegetable_images/gaussian_noise/test
length of original dataset: 15000
1500
